In [12]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

In [3]:
from tensorflow.keras.layers import Input, Add, GlobalAveragePooling2D, Dense, Activation, Conv2D
from tensorflow.keras import layers, Model

def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x
    
    # Shortcut path
    if x.shape[-1] != filters:
        shortcut = Conv2D(filters, (1,1), strides=stride, padding="same")(x)
        shortcut = layers.BatchNormalization()(shortcut)
    
    # First convolution layer
    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    # Second convolution layer
    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding="same")(x)
    x = layers.BatchNormalization()(x)
    
    # Adding the shortcut to the output
    x = Add()([x, shortcut])
    x = layers.Activation("relu")(x)
    
    return x

def create_resnet_model(input_shape, num_classes):
    input = Input(shape=input_shape)
    x = layers.Conv2D(64, (3,3), padding="same")(input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = residual_block(x, 64)
    x = residual_block(x, 128)
    x = residual_block(x, 256)

    x = GlobalAveragePooling2D()(x)
    x = layers.Dense(2048, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(2048, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    output = layers.Dense(num_classes, activation='sigmoid')(x)
    model = Model(inputs=input, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC'])
    
    return model


In [4]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 이미지 데이터 폴더 경로
train_data_folder = r'C:\Users\pc\Desktop\CNN\Train_Image'
val_data_folder = r'C:\Users\pc\Desktop\CNN\Validation_Image'
test_data_folder = r'C:\Users\pc\Desktop\CNN\Test_Image'

# 모델 생성
input_shape = (28, 28, 1)  # 입력 이미지의 크기와 채널 수
num_classes = 1  # 이진 분류 문제의 경우 클래스 개수는 1
model = create_resnet_model(input_shape, num_classes)

# ImageDataGenerator 생성
datagen = ImageDataGenerator(rescale=1./255)

# 학습 데이터 로드
train_generator = datagen.flow_from_directory(
    train_data_folder,
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=32,
    shuffle=True
)

# Validation 데이터 로드
val_generator = datagen.flow_from_directory(
    val_data_folder,
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=32,
    shuffle=False  # Validation 데이터는 섞지 않음
)

# Test 데이터 로드
test_generator = datagen.flow_from_directory(
    test_data_folder,
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=32,
    shuffle=False  # Test 데이터는 섞지 않음
)

# 모델 학습
epochs = 50
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)

# 모델 평가
test_loss, test_accuracy, test_auc = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
print("Test AUC:", test_auc)

Found 200000 images belonging to 2 classes.
Found 24000 images belonging to 2 classes.
Found 24000 images belonging to 2 classes.
Epoch 1/50
6250/6250 [==============================] - 3387s 542ms/step - loss: 0.5038 - accuracy: 0.7999 - auc: 0.5132 - val_loss: 0.4506 - val_accuracy: 0.8333 - val_auc: 0.5267
Epoch 2/50
6250/6250 [==============================] - 3215s 514ms/step - loss: 0.5007 - accuracy: 0.8000 - auc: 0.5188 - val_loss: 0.4560 - val_accuracy: 0.8333 - val_auc: 0.5431
Epoch 3/50
6250/6250 [==============================] - 3155s 505ms/step - loss: 0.4991 - accuracy: 0.8000 - auc: 0.5383 - val_loss: 0.4487 - val_accuracy: 0.8333 - val_auc: 0.5554
Epoch 4/50
6250/6250 [==============================] - 3179s 509ms/step - loss: 0.4979 - accuracy: 0.8000 - auc: 0.5513 - val_loss: 0.4573 - val_accuracy: 0.8333 - val_auc: 0.5573
Epoch 5/50
6250/6250 [==============================] - 3457s 553ms/step - loss: 0.4975 - accuracy: 0.8000 - auc: 0.5552 - val_loss: 0.4527 - val_

KeyboardInterrupt: 